In [1]:
import os
os.environ['OMP_NUM_THREADS'] = "16"
os.environ['OMP_DISPLAY_ENV'] = "True"
os.environ['OMP_PROC_BIND'] = "True"
os.environ['OMP_SCHEDULE'] = "STATIC"

In [2]:
import time
import logging
import numpy as np
import torch
import torch.nn as nn
import torch._dynamo
import torch._inductor
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torch.utils.data.sampler import SubsetRandomSampler
from torch.profiler import profile, record_function, ProfilerActivity


OPENMP DISPLAY ENVIRONMENT BEGIN
  _OPENMP = '201511'
  OMP_DYNAMIC = 'FALSE'
  OMP_NESTED = 'FALSE'
  OMP_NUM_THREADS = '16'
  OMP_SCHEDULE = 'STATIC'
  OMP_PROC_BIND = 'TRUE'
  OMP_PLACES = '{0},{1},{2},{3},{4},{5},{6},{7},{8},{9},{10},{11},{12},{13},{14},{15},{16},{17},{18},{19},{20},{21},{22},{23},{24},{25},{26},{27},{28},{29},{30},{31}'
  OMP_STACKSIZE = '14398978'
  OMP_WAIT_POLICY = 'PASSIVE'
  OMP_THREAD_LIMIT = '4294967295'
  OMP_MAX_ACTIVE_LEVELS = '2147483647'
  OMP_CANCELLATION = 'FALSE'
  OMP_DEFAULT_DEVICE = '0'
  OMP_MAX_TASK_PRIORITY = '0'
OPENMP DISPLAY ENVIRONMENT END


In [3]:
#torch.set_num_threads(16)
#torch.set_num_interop_threads(1)
print(torch.__config__.parallel_info())

ATen/Parallel:
	at::get_num_threads() : 16
	at::get_num_interop_threads() : 16
OpenMP 201511 (a.k.a. OpenMP 4.5)
	omp_get_max_threads() : 16
Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
	mkl_get_max_threads() : 16
Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
std::thread::hardware_concurrency() : 32
Environment variables:
	OMP_NUM_THREADS : 16
	MKL_NUM_THREADS : [not set]
ATen parallel backend: OpenMP



In [4]:
def build_data_loader(data_dir, batch_size, random_seed=42, valid_size=0.1, shuffle=True, test=False):

    transform = transforms.Compose([transforms.ToTensor()])
    transform.crop_size=224
    transform.resize_size=224
    
    train_dataset = datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform)
    test_dataset = datasets.CIFAR10(root=data_dir, train=False, download=True, transform=transform)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)

    return (train_loader, test_loader)

In [5]:
def train(model, data_loader, num_epochs, criterion, optimizer, device):
    total_steps = len(train_loader)
    for epoch in range(num_epochs):
        start = time.time()
        for step, (images, labels) in enumerate(train_loader):  
            # Move tensors to the configured device
            images = images.to(device)
            labels = labels.to(device)
        
            # Forward pass
            outputs = model(images)
            outputs.to(device)
            loss = criterion(outputs, labels)
        
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #print ('Step [{}/{}], Loss: {:.4f}'.format(step+1, total_steps, loss.item()))
        end = time.time()
               
        print ('Epoch [{}/{}], Loss: {:.4f}, time: {} seconds'.format(epoch+1, num_epochs, loss.item(), int(end-start)))
            

In [6]:
def test(model, test_loader, device):
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))  

In [7]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))

        self.fc1 = nn.Linear(4096, 512)
        
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [8]:
# General parameters
data_dir = '/tmp'
device = "cpu"

# Hyperparameters
lr = 0.00001
weight_decay = 0.005
batch_size = 64
num_epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam

# FashionMNIST dataset 
train_loader, test_loader = build_data_loader(data_dir=data_dir, batch_size=batch_size)

# Models
#weights = models.VGG11_Weights.DEFAULT
#net = models.vgg11(weights=weights)
#net.train()
model = CNN().to(device)

# Optimizer
optimizer = optimizer(model.parameters(), lr, weight_decay=weight_decay)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
start = time.time()
train(model, train_loader, num_epochs, criterion, optimizer, device)
end = time.time()
print('Training time: {} seconds'.format(int(end-start)))
test(model, test_loader, device)

Epoch [1/10], Loss: 2.1906, time: 10 seconds
Epoch [2/10], Loss: 2.2061, time: 10 seconds
Epoch [3/10], Loss: 2.1815, time: 12 seconds
Epoch [4/10], Loss: 2.1481, time: 12 seconds
Epoch [5/10], Loss: 2.1192, time: 12 seconds
Epoch [6/10], Loss: 2.0960, time: 12 seconds
Epoch [7/10], Loss: 2.0759, time: 12 seconds
Epoch [8/10], Loss: 2.0573, time: 10 seconds
Epoch [9/10], Loss: 2.0388, time: 10 seconds
Epoch [10/10], Loss: 2.0218, time: 10 seconds
Training time: 114 seconds
Accuracy of the network on the 10000 test images: 47.86 %


In [10]:
#schedule = torch.profiler.schedule(wait=1, warmup=3, active=5, repeat=5)
#activities = [ProfilerActivity.CPU, ProfilerActivity.CUDA]
activities = [ProfilerActivity.CPU]
prof = profile(activities=activities)
        
input_sample, _ = next(iter(train_loader))

prof.start()
#train(model, train_loader, num_epochs, criterion, optimizer, device)
prof.stop()

#prof.export_chrome_trace("./cnn_trace.json")
print(prof.key_averages().table(sort_by="self_cpu_time_total", row_limit=10))

STAGE:2023-06-25 12:21:17 566721:566721 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-06-25 12:21:17 566721:566721 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-06-25 12:21:17 566721:566721 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
